<a href="https://colab.research.google.com/github/gregorio-saporito/Spark-AMD/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding similar items: StackSample
Gregorio Luigi Saporito - DSE (2020-2021)

### Upload to session storage the Kaggle API token

In [1]:
from google.colab import files
uploaded = files.upload()

import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

Saving kaggle.json to kaggle.json


### Download the dataset through the Kaggle API

In [2]:
# access permissions with the API token
!chmod 600 /content/kaggle.json
!kaggle datasets download -d stackoverflow/stacksample
!unzip \*.zip && rm *.zip
# remove datasets which are not needed
!rm Answers.csv
!rm Tags.csv

100% 1.10G/1.11G [00:34<00:00, 26.8MB/s]
100% 1.11G/1.11G [00:34<00:00, 34.9MB/s]
Archive:  stacksample.zip
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


### Spark environment setup

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz 
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!rm /content/spark-3.1.1-bin-hadoop2.7.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

import findspark
findspark.init("spark-3.1.1-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

import pyspark
type(spark)

sc = spark.sparkContext

### Load Dataset
Spark reads files line by line for performance reasons and CSVs with newline characters cause problems for the parser. In this case the Body column of the file "Questions.csv" has characters like "\n" and "\r" which compromise the correct loading of the dataset. To solve the problem a third party parser capable of coping with this issue was used and a .csv file without newline characters is written on Disk. An alternative in a production scenario would be storing the files in a database. The RAM used for the parser is then freed up to save space. The new .csv file is then correctly loaded with Spark.

In [4]:
import pandas as pd
parsed = pd.read_csv("Questions.csv", encoding="ISO-8859-1", usecols=["Body"])
parsed['Body'] = parsed['Body'].str.replace(r'\n|\r', '')
parsed.to_csv("Body.csv")
del parsed

In [5]:
df = spark.read.load("Body.csv", format="csv",
                     inferSchema="true", header="true")
df

DataFrame[_c0: int, Body: string]

In [6]:
df.show(10)

+---+--------------------+
|_c0|                Body|
+---+--------------------+
|  0|"<p>I've written ...|
|  1|"<p>Are there any...|
|  2|<p>Has anyone got...|
|  3|<p>This is someth...|
|  4|"<p>I have a litt...|
|  5|<p>I am working o...|
|  6|<p>I've been writ...|
|  7|"<p>I wonder how ...|
|  8|<p>I would like t...|
|  9|<p>I'm trying to ...|
+---+--------------------+
only showing top 10 rows



### Dataset Cleaning

In [7]:
# check for missing values
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+----+
|_c0|Body|
+---+----+
|  0|   0|
+---+----+



In [8]:
from pyspark.sql.functions import col, lower, regexp_replace, split

def clean_html(x):
  x = regexp_replace(x, '<.*?>', '')
  return x

df = df.select(col("_c0").alias("id"), clean_html(col("Body")).alias("Body"))

In [9]:
df.show(10)

+---+--------------------+
| id|                Body|
+---+--------------------+
|  0|"I've written a d...|
|  1|"Are there any re...|
|  2|Has anyone got ex...|
|  3|This is something...|
|  4|"I have a little ...|
|  5|I am working on a...|
|  6|I've been writing...|
|  7|"I wonder how you...|
|  8|I would like the ...|
|  9|I'm trying to mai...|
+---+--------------------+
only showing top 10 rows



In [10]:
# extracting tokens from text
from pyspark.ml.feature import RegexTokenizer

regexTokenizer = RegexTokenizer(gaps = False, pattern = '\w+', inputCol = 'Body', outputCol = 'tokens')
tokenised = regexTokenizer.transform(df)
tokenised.show(3)

+---+--------------------+--------------------+
| id|                Body|              tokens|
+---+--------------------+--------------------+
|  0|"I've written a d...|[i, ve, written, ...|
|  1|"Are there any re...|[are, there, any,...|
|  2|Has anyone got ex...|[has, anyone, got...|
+---+--------------------+--------------------+
only showing top 3 rows



In [11]:
# stopwords removal
from pyspark.ml.feature import StopWordsRemover
swr = StopWordsRemover(inputCol = 'tokens', outputCol = 'sw_removed')
Body_swr = swr.transform(tokenised)
Body_swr.show(3)

+---+--------------------+--------------------+--------------------+
| id|                Body|              tokens|          sw_removed|
+---+--------------------+--------------------+--------------------+
|  0|"I've written a d...|[i, ve, written, ...|[ve, written, dat...|
|  1|"Are there any re...|[are, there, any,...|[really, good, tu...|
|  2|Has anyone got ex...|[has, anyone, got...|[anyone, got, exp...|
+---+--------------------+--------------------+--------------------+
only showing top 3 rows



### Jaccard Distance Approach

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, MinHashLSH
import pyspark.sql.functions as f
from pyspark.sql.functions import monotonically_increasing_id 

jd_df = Body_swr\
  .withColumn("nodup", f.array_distinct("sw_removed"))\
  .select("id","nodup")
  #.withColumn("id", col("id").cast("double"))

In [13]:
jd_df.show(3)

+---+--------------------+
| id|               nodup|
+---+--------------------+
|  0|[ve, written, dat...|
|  1|[really, good, tu...|
|  2|[anyone, got, exp...|
+---+--------------------+
only showing top 3 rows



In [14]:
jd_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- nodup: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [34]:
from pyspark.sql.functions import col, size
# filer out empty arrays of strings and on a smaller chunk of the dataset
jd_df_clean = jd_df.where((size(col("nodup")) >= 1) & (col("id") <= 10000))

In [35]:
import time
start = time.time()

model = Pipeline(stages=[
        HashingTF(inputCol="nodup", outputCol="vectors"),
        MinHashLSH(inputCol="vectors", outputCol="lsh", numHashTables=5)
    ]).fit(jd_df_clean)

db_hashed = model.transform(jd_df_clean)

print("Approximately joining on distance smaller than 0.5:")
db_matches = model.stages[-1].approxSimilarityJoin(db_hashed, db_hashed, 0.5)

#show all matches (including duplicates)
db_matches.select(f.col('datasetA.id').alias('id_A'),
                 f.col('datasetB.id').alias('id_B'),
                 f.col('distCol')).show()

end = time.time()
print(end-start)

Approximately joining on distance smaller than 0.5:
+-----+-----+-------+
| id_A| id_B|distCol|
+-----+-----+-------+
|24433|24433|    0.0|
|50667|50667|    0.0|
| 3589| 3589|    0.0|
|25075|25075|    0.0|
|24522|24522|    0.0|
|30607|30607|    0.0|
| 2340| 2340|    0.0|
|56529|56529|    0.0|
|19687|19687|    0.0|
|  110|  110|    0.0|
| 7935| 7935|    0.0|
| 9749| 9749|    0.0|
|10651|10651|    0.0|
|16394|16394|    0.0|
|22267|22267|    0.0|
|23404|23404|    0.0|
|29919|29919|    0.0|
|35802|35802|    0.0|
|36257|36257|    0.0|
|37731|37731|    0.0|
+-----+-----+-------+
only showing top 20 rows

2073.4282355308533


In [17]:
#show non-duplicate matches
db_matches.select(f.col('datasetA.id').alias('id_A'),
                 f.col('datasetB.id').alias('id_B'),
                 f.col('distCol')).filter('id_A < id_B').show()

KeyboardInterrupt: ignored

In [ ]:
# example filter on ids 6520 and 6522 where jaccard distance is 0.48
df.where((col("id") == 6520) | (col("id") == 6522)).show(truncate=False)

In [ ]:
# example filter on ids 9514 and 9919 where jaccard distance is 0.06
df.where((col("id") == 9514) | (col("id") == 9919)).show(truncate=False)